## Import Libraries

In [1]:
!pip install XlsxWriter

In [2]:
from urllib.request import Request, urlopen
from datetime import datetime
import re
import json
import pandas as pd

## Read all countries listed

In [3]:
countries = pd.read_excel("all_countries_listed.xlsx")

In [4]:
base_url = "https://www.passportindex.org/passport/"
count = 0
all_data = []
all_countries = []
for _ ,row in countries.iterrows():
    try:
        # Get country name and slug
        country = row['name']
        slug = row['slug']

        # Build URL and fetch webpage
        url = (base_url+slug).replace(" ", "")
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read().decode("utf8")

        # Extract data from webpage
        finder = re.findall(r'sample_data = .*;', webpage)
        data = finder[0].replace('sample_data = ', '').replace(';','').strip()
        data = data.replace('\t','')
        data = data.replace('\n','')
        data = data.replace(',}','}')
        data = data.replace(',]',']')
        data = json.loads(data)

        # Append country data to list
        all_data.append({"country": country,"data":data})
        all_countries.append(country)
        count = count + 1
    except:
        print("Data not found for", country)
        continue


Data not found for Anguilla
Data not found for Antarctica
Data not found for Aruba
Data not found for Bermuda
Data not found for Bouvet Island
Data not found for British Indian Ocean Territory
Data not found for British Virgin Islands
Data not found for Cayman Islands
Data not found for Christmas Island
Data not found for Cocos [Keeling] Islands
Data not found for Cook Islands
Data not found for Falkland Islands [Islas Malvinas]
Data not found for Faroe Islands
Data not found for French Guiana
Data not found for French Polynesia
Data not found for French Southern Territories
Data not found for Gaza Strip
Data not found for Gibraltar
Data not found for Greenland
Data not found for Guadeloupe
Data not found for Guam
Data not found for Guernsey
Data not found for Heard Island and McDonald Islands
Data not found for Isle of Man
Data not found for Jersey
Data not found for Martinique
Data not found for Mayotte
Data not found for Montserrat
Data not found for Netherlands Antilles
Data not fo

In [5]:
all_df = []

#iterate through all data and create a dataframe
for item in all_data:
    df = pd.DataFrame([item['data']]).T.reset_index()
    df['country'] = item['country']
    all_df.append(df)

data_df = pd.concat(all_df, ignore_index=True)
data_df['index'] = data_df['index'].str.upper()
data_df = data_df.rename(columns={'index': 'country_code',
                                  'country': 'origin',
                                  0:'requirement_status'
                                  })
data_df

,country_code,requirement_status,origin
0,AL,0,Afghanistan
1,DZ,0,Afghanistan
2,AD,0,Afghanistan
3,AO,1,Afghanistan
4,AG,0,Afghanistan
...,...,...,...
39596,VE,0,Zimbabwe
39597,VN,0,Zimbabwe
39598,YE,0,Zimbabwe
39599,ZM,2,Zimbabwe


In [6]:
#Merge country code with country name
data_df['destination']= data_df[['country_code']].merge(countries,how='left').name
data_df.loc[data_df['country_code'] == 'NA', 'destination'] = 'Namibia'

#Map requirement status to readable text
data_df = data_df[['origin', 'destination', 'requirement_status']]
mapping = {'0': 'visa_required', '1': 'visa_on_arrival', '2': 'visa_free', '3': 'eta'}
data_df['requirement'] = data_df['requirement_status'].map(mapping)

data_df

,origin,destination,requirement_status,requirement
0,Afghanistan,Albania,0,visa_required
1,Afghanistan,Algeria,0,visa_required
2,Afghanistan,Andorra,0,visa_required
3,Afghanistan,Angola,1,visa_on_arrival
4,Afghanistan,Antigua and Barbuda,0,visa_required
...,...,...,...,...
39596,Zimbabwe,Venezuela,0,visa_required
39597,Zimbabwe,Vietnam,0,visa_required
39598,Zimbabwe,Yemen,0,visa_required
39599,Zimbabwe,Zambia,2,visa_free


In [7]:
country_lst = []
visa_free_lst = []
visa_on_arrival_lst = []
eta_lst = []
visa_required_lst = []
mobility_score_lst = []

for country in all_countries:
    country_status = data_df[data_df.origin == country].requirement.value_counts()
    visa_free = country_status.get('visa_free', 0)
    visa_on_arrival = country_status.get('visa_on_arrival', 0)
    eta_val = country_status.get('eta', 0)
    visa_required = country_status.get('visa_required', 0)

    mobility_score = visa_free + visa_on_arrival + eta_val

    country_lst.append(country)
    visa_free_lst.append(visa_free)
    visa_on_arrival_lst.append(visa_on_arrival)
    eta_lst.append(eta_val)
    visa_required_lst.append(visa_required)
    mobility_score_lst.append(mobility_score)

data={'country':country_lst,
      'visa_free':visa_free_lst,
      'visa_on_arrival':visa_on_arrival_lst,
      'eta (Electronic travel authorization)': eta_lst,
      'visa_required': visa_required_lst,
      'mobility_score':mobility_score_lst

      }
df_country_status = pd.DataFrame(data).sort_values(by='mobility_score', ascending=False).reset_index(drop=True)

In [8]:
#Save to excel
data_df.index += 1
df_country_status.index += 1
today_date = datetime.today().strftime('%Y-%m-%d')
file_name = "passport-power"+"-"+today_date+".xlsx"

writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

data_df.to_excel(writer, sheet_name='Sheet 1')
df_country_status.to_excel(writer, sheet_name='Sheet 2')

writer.save()

In [9]:
print("Total Countries: ", len(all_data))

Total Countries:  199


In [10]:
data_df[data_df['destination'].isnull()]

,origin,destination,requirement_status,requirement


In [11]:
# Check status for Nepal
status = ["visa_required"]
data_df[(data_df["origin"] == "Nepal") & (data_df['requirement'].isin(status))]

,origin,destination,requirement_status,requirement
24478,Nepal,Afghanistan,0,visa_required
24479,Nepal,Albania,0,visa_required
24480,Nepal,Algeria,0,visa_required
24481,Nepal,Andorra,0,visa_required
24483,Nepal,Antigua and Barbuda,0,visa_required
...,...,...,...,...
24671,Nepal,Venezuela,0,visa_required
24672,Nepal,Vietnam,0,visa_required
24673,Nepal,Yemen,0,visa_required
24674,Nepal,Zambia,0,visa_required
